# Genome Informatics 2021 - Project assignment
## Finding intersection between VCF files


The purpose of this notebook is to demonstrate a method of comparing VCF files of three sample genomes (a mother, a father and a child) in order to find *de novo* variants of the child. *De novo* variants represent the variants present in the child, but not present in either of the parents.

The VCF files used in this notebook were obtained via GRAF Germline Variant Detection Workflow, developed by Seven Bridges.

*PyVCF* library is being utilized in order to easily parse through the VCF files.
It has built-in functions for reading and writing into VCF files, as well as direct ways of accessing the records and their fields.

In [1]:
!pip install pyvcf
import vcf

  Created wheel for pyvcf: filename=PyVCF-0.6.8-cp38-cp38-linux_x86_64.whl size=162299 sha256=e220170903b5ed7ea4ba25ccf9e67e12effc1e937cc72cb7382165040580edff
  Stored in directory: /home/jovyan/.cache/pip/wheels/f0/3e/15/c1865f9a071eacc7fc702e9c7d25fc752fe5e30a82d0dd405d
Successfully built pyvcf


In [2]:
# Open all VCF files in read mode
vcf_child = vcf.Reader(open('/sbgenomics/project-files/vcf_ch38/HG002-NA24385-50x_filtered_corr.vcf', 'r'))
vcf_father = vcf.Reader(open('/sbgenomics/project-files/vcf_ch38/HG003.hs37d5.60x.1.converted_filtered_corr.vcf', 'r'))
vcf_mother = vcf.Reader(open('/sbgenomics/project-files/vcf_ch38/HG004.hs37d5.60x.1.converted_filtered_corr.vcf', 'r'))


Let's define a function in order to calculate the number and percentage of *de novo* variants from two VCF files.
Input arguments represent a baseline VCF file, a VCF file containing called variants as well as the output file name.
Output values represent a list containing the number of total and de novo variants.

In [3]:
def vcf_compare(baseline_vcf, call_vcf, out_file_name):
    # Initialize number of total and de novo variants to 0 
    de_novo_num = 0
    total_variants = 0

    # Open the output file in write mode and write the header section into the output file
    out_file_vcf = vcf.Writer(open(out_file_name+'.vcf', 'w'), call_vcf)  #/sbgenomics/project-files/vcf_ch38/'+out_file_name+'.vcf', 'w'), call_vcf) 

    # Initialize baseline record and position
    baseline_record = next(baseline_vcf)
    baseline_position = baseline_record.POS

    # Main loop
    for call_record in call_vcf: 
        # Update positions
        call_position = call_record.POS
        baseline_position = baseline_record.POS

        total_variants = total_variants + 1

        if baseline_record.CHROM == call_record.CHROM: # If we are examining records on the same chromosome, compare their variants
            [baseline_record, total_change, de_novo_change] = chrom_compare(call_record, baseline_record, baseline_vcf, call_vcf, out_file_vcf)

        # Different chromosome, baseline record on the next chromosome. If the variant in the call record is of adequate quality
        # and is not a false positive, it is classified as a de novo variant, since the are no more variants on the current chromosome
        # in the baseline VCF
        elif call_position > baseline_position:
            if len(call_record.FILTER) == 1 or call_record.QUAL <= 10:
                # Skip this record and substract if from the total number of variants
                total_variants = total_variants - 1
            else:
                # De novo variant detected
                de_novo_num = de_novo_num + 1 
                # Copy the record into the output file
                out_file_vcf.write_record(call_record)

        # Different chromosome, call record on the next chromosome. Iterate through the baseline VCF until the baseline record 
        # chromosome matches the call record chromosome, then compare those variants
        else: # call_position < baseline_position:
            while call_record.CHROM != baseline_record.CHROM:
                baseline_record = next(baseline_vcf)
            [baseline_record, total_change, de_novo_change] = chrom_compare(call_record, baseline_record, baseline_vcf, call_vcf, out_file_vcf)
                    
        # Calculate the new number of total and de novo variants
        de_novo_num = de_novo_num + de_novo_change
        total_variants = total_variants + total_change

    out_file_vcf.close()

    return [de_novo_num, total_variants]


We also have to define an additional function that performs the comparison for the variants on the same chromosome. Input arguments are the current call and baseline records and all three VCF files. The function returns the change in the numbers of total and *de novo* variants.

In [4]:
def chrom_compare (call_record, baseline_record, baseline_vcf, call_vcf, out_file_vcf):
    # Initialize the updated values and positions
    de_novo_change = 0
    total_change = 0
    call_position = call_record.POS
    baseline_position = baseline_record.POS

    if len(call_record.FILTER) == 1 or call_record.QUAL <= 10:
        # False positive or low quality -> skip this record
        total_change = -1

    # Baseline record ahead of call record -> call record doesn't have a match -> de novo
    elif call_position < baseline_position:
        de_novo_change = 1  
        # Copy the record into the output file
        out_file_vcf.write_record(call_record)

    # Call record ahead of baseline record -> iterate through baseline VCF until the baseline position
    # catches up to the call record position
    else:
        try:
            while call_position > baseline_position:
                baseline_record = next(baseline_vcf)
                baseline_position = baseline_record.POS
                # If the baseline record reaches the next chromosome, stop iterating 
                if baseline_record.CHROM != call_record.CHROM:
                    break

            # If variants on the same position are detected, compare REF and ALT fields, as well as their genotype
            if baseline_position == call_position:
                
                base_gt = baseline_record.genotype(baseline_vcf.samples[0])['GT']
                call_gt = call_record.genotype(call_vcf.samples[0])['GT']
                base_ref = baseline_record.REF
                call_ref = call_record.REF
                base_alt = baseline_record.ALT
                call_alt = call_record.ALT
                
                # Compare the genotypes of call and baseline, and determin if the record is a de novo variant
                if gt_compare(base_gt, call_gt, base_ref, call_ref, base_alt, call_alt):
                    # Same mutation -> not a de novo variant -> skip
                    pass
                else:
                    # De novo variant detected
                    de_novo_change = 1  
                    # Copy the record into the output file
                    out_file_vcf.write_record(call_record)

            # The baseline position is ahead of the call position -> call position has no match -> de novo 
            else:
                # De novo variant detected
                de_novo_change = 1  
                # Copy the record into the output file
                out_file_vcf.write_record(call_record)

        # Try/except branches exist in case the baseline_record reaches the end of the baseline VCF file. In that case,
        # the current and all the following variants from the call VCF are detected as de novo variants
        except:
            # De novo variant detected
            de_novo_change = 1 
            # Copy the record into the output file
            out_file_vcf.write_record(call_record)

    return [baseline_record, total_change, de_novo_change]

We have defined an additional function for comparing the REF and ALT fields depending on the genotypes of the call and baseline records.

In [5]:
def gt_compare(base_gt, call_gt, base_ref, call_ref, base_alt, call_alt):
    # Return flag=1 if call haplotypes are a mutation that matches a parent haplotype or are not a mutation at all
    
    if call_gt == '0/0':
        flag = 1
    elif call_gt == '0/1' and base_gt == '0/0':
        flag = 0
    elif call_gt == '0/1' and base_gt in ['0/1','1/1']:
        if call_ref == base_ref and call_alt == base_alt:
            flag = 1
        else:
            flag = 0
    elif call_gt == '0/1' and base_gt == '1/2':
        if call_ref == base_ref and call_alt in base_alt:
            flag = 1
        else:
            flag = 0 
    elif call_gt == '1/1' and base_gt == '0/0':
        flag = 0
    elif call_gt == '1/1' and base_gt in ['0/1','1/1']:
        if call_ref == base_ref and call_alt == base_alt:
            flag = 1
        else:
            flag = 0
    elif call_gt == '1/1' and base_gt == '1/2':
        if call_ref == base_ref and call_alt in base_alt:
            flag = 1
        else:
            flag = 0
    elif call_gt == '1/2' and base_gt == '0/0':
        flag = 0
    elif call_gt == '1/2' and base_gt in ['0/1','1/1']:
        if call_ref == base_ref and (base_alt in call_alt):
            flag = 1
        else:
            flag = 0
    elif call_gt == '1/2' and base_gt == '1/2':
        if call_ref == base_ref and (call_alt[0] in base_alt or call_alt[1] in base_alt):
            flag = 1
        else:
            flag = 0
    
    return flag

The main section of the code calls the *vcf_compare* function twice. The first time, it compares the child VCF against the mother VCF. The output of this comparison is then compared against the father VCF, giving the final VCF containing all the *de novo* variants.

In [6]:
# Compare the child VCF with the mother VCF and find de novo variants
[num_m_c, total_m_c] = vcf_compare(vcf_mother, vcf_child, 'de_novo_mother_child')

total_child_variants = total_m_c

# Open the created VCF containing de novo variants detected between the mother and the child
vcf_mother_child = vcf.Reader(open('de_novo_mother_child.vcf', 'r')) #'/sbgenomics/project-files/vcf_ch38/de_novo_mother_child.vcf', 'r'))

# Find de novo variants between the father VCF and the created file
[num_final, total_final] = vcf_compare(vcf_father, vcf_mother_child, 'de_novo_final')

# Print the total number of and the percentage of de novo variants 
print("Total number of de novo variants: ", num_final)
print("Percentage of de novo variants: ", num_final/total_child_variants*100, "%")

Total number of de novo variants:  142685
Percentage of de novo variants:  2.90858381312005 %
